In [ ]:
import xgboost as xgb
import inspect

print(inspect.getfile(xgb.train))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier
import xgboost as xgb
from dataset.load_dat import load_keel_dat
from kmeans_smote import KMeansSMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import numpy as np

# **加载数据集（以 glass1 数据集为例）**
def load_keel(name):
    # 从 UCI 或本地加载 'glass1' 数据集
    # 下载地址: https://sci2s.ugr.es/keel/dataset/data/classification/glass1.zip
    # 假设数据已经保存为 glass1.csv
    data = load_keel_dat("dataset/"+name)  # 替换为实际路径
    print(data)
    X = data.iloc[:, :-1].values  # 特征
    y = data.iloc[:, -1].values   # 标签
    return X, y

# 主流程（K 折）
def main():
    #name = "abalone19.dat" #scale_pos_weight=0.1
    name = "yeast6.dat"
    X, y = load_keel(name)
    print(X.shape)
    print(y)


In [1]:
import pandas as pd
from sdv.single_table import CTGANSynthesizer, TVAESynthesizer
from sdv.metadata import SingleTableMetadata
import torch

def generate_synthetic_samples(X, y, class_value=1, n_samples=500, method='ctgan', epochs=300):

    # 检查 GPU
    has_cuda = torch.cuda.is_available()
    print(f"🚀 GPU 可用: {has_cuda}")

    # 转 DataFrame
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X, columns=[f'f{i}' for i in range(X.shape[1])])
    y = pd.Series(y, name='label')

    # 获取目标类数据
    real_data = X.copy()
    real_data['label'] = y
    real_data = real_data[real_data['label'] == class_value].reset_index(drop=True)

    # Metadata
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(real_data)

    # 初始化模型
    if method.lower() == 'ctgan':
        model = CTGANSynthesizer(
            metadata,
            epochs=epochs,
            verbose=True,
            cuda=has_cuda  # ✅ 正确方式
        )
    elif method.lower() == 'tvae':
        if has_cuda:
            print("⚠️ TVAE 暂不支持 GPU，将使用 CPU")
        model = TVAESynthesizer(
            metadata,
            epochs=epochs,
            verbose=True
        )
    else:
        raise ValueError("method must be 'ctgan' or 'tvae'")

    # 拟合模型
    print(f"📦 开始训练 {method.upper()} 模型，共 {epochs} 轮")
    model.fit(real_data)
    print("✅ 模型训练完成")

    # 合成数据
    condition = {'label': class_value}
    synthetic_data = model.sample(num_rows=n_samples, conditions=[condition])

    synthetic_X = synthetic_data.drop(columns='label')
    synthetic_y = synthetic_data['label']

    return synthetic_X, synthetic_y, synthetic_data, real_data, metadata

In [ ]:
from sklearn.datasets import make_classification
from sdv.evaluation.single_table import evaluate_quality

# 模拟数据
X, y = make_classification(n_samples=1000, n_features=20, weights=[0.95, 0.05], random_state=42)

# 生成异常类（class 1）样本
synthetic_X, synthetic_y, synthetic_data, real_data, metadata = generate_synthetic_samples(
    X, y, class_value=1, n_samples=500, method='ctgan', epochs=200
)

# SDV 质量评估
score = evaluate_quality(real_data=real_data, synthetic_data=synthetic_data, metadata=metadata)
print("📊 SDV 质量评分:", round(score, 4))

🚀 GPU 可用: False
📦 开始训练 CTGAN 模型，共 200 轮
